In [1]:
import os as os
import seaborn as sns
import pandas as pd

os.chdir('C:/Users/falco/Desktop/directory/WMMEAS/')
data_path = 'C:/Users/falco/Desktop/directory/WMMEAS data/'
#from WRMDpy import WRMDalert as alert
from WRMDpy import read_data as rd
from WRMDpy import Syndrome as Syndrome
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
plt.style.use('seaborn-whitegrid')
sns.set_palette("husl", 5)
plt.rcParams['figure.figsize'] = (12, 12)
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['axes.labelsize'] = plt.rcParams['font.size']
plt.rcParams['axes.titlesize'] = 1.5*plt.rcParams['font.size']
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size']
plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']
plt.rcParams['xtick.major.size'] = 3
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['legend.frameon'] = False
plt.rcParams['legend.loc'] = 'center left'
plt.rcParams['axes.linewidth'] = 1
#plt.style.use('fivethirtyeight')

C:\Users\falco\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


AttributeError: module 'sklearn.metrics' has no attribute 'jaccard_similarity_score'

In [ ]:
%%time
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import
from sklearn.model_selection import StratifiedKFold ,cross_val_score, train_test_split, cross_val_predict
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve
from pandas_ml import ConfusionMatrix
from textblob import TextBlob
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
%%time
data = Syndrome.read_data(data_path+'\Training dataset_WMME_May 7 2018.csv')
data.shape

In [ ]:
data.head()

In [ ]:
len(data.condition_predict.unique())

In [ ]:
data.condition_predict.iloc[0]

In [ ]:
data.Condition.iloc[0]

In [ ]:
%%time
h_data = Syndrome.read_data_historic(data_path+'\historic_data_2013-01-01_2018-05-22_above_all major centers.csv')
h_data.shape

In [ ]:
h_data.Admitted_at.max()

In [ ]:
h_data.Admitted_at.min()

In [ ]:
def split_into_tokens(message):
    message = unicode(message, 'utf8')  # convert bytes into proper unicode
    return TextBlob(message).words


def split_into_lemmas(message):
    message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]


def bag_of_words(data_frame, historical_data):
    X_train, X_test, y_train, y_test = train_test_split(data_frame['Condition'], data_frame['condition_predict'], test_size=0.2, random_state=0)
    
    count_vect = CountVectorizer(analyzer=split_into_lemmas,ngram_range= (1, 3), encoding='utf8',stop_words =None)
    X_train_counts = count_vect.fit_transform(X_train)
    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
    print('Training shape')
    print (X_train_tfidf.shape)
    
    X_new_counts = count_vect.transform(X_test)
    X_test_tfidf = tfidf_transformer.transform(X_new_counts)

    print ('Testing shape')
    print (X_test_tfidf.shape)
    
    
    print ('Historical data')
    X_hist_counts = count_vect.transform(historical_data['ConditionO'])
    X_hist_tfidf = tfidf_transformer.transform(X_hist_counts)
    print (X_hist_tfidf.shape)

    
    return X_train_tfidf, y_train, X_test_tfidf, y_test, X_hist_tfidf, count_vect

In [ ]:
%%time
X_train_tfidf, y_train, X_test_tfidf, y_test, X_hist_tfidf, count_vectrotize_obj = bag_of_words(data_frame= data, historical_data= h_data)

In [ ]:
X_train_tfidf

In [ ]:
%%time
features = count_vectrotize_obj.get_feature_names()

In [ ]:
%%time

## Set the parameters by cross-validation
#tuned_parameters = [{'n_estimators': [10, 100],
#           'max_features': ['auto', 'sqrt', 'log2'],
#           'max_depth' : [3, 6],
#           'criterion' :['gini', 'entropy']}]

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': [1e-3, 1e-4, 1e-3, 1e-5],
                     'C': [1, 10, 100, 100, 1000], 
                    'probability':[True]},
                   
                    {'kernel': ['linear'], 'C': [0.25,1, 10, 100, 1000], 'probability':[True]}, 
                    {'kernel': ['poly'], 'C': [0.25, 1, 10, 100, 1000], 'probability':[True], 
                     'degree':[1,3,5], 'gamma':[1,2, 4,8]}
                   ]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    
    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score, verbose = 1,  n_jobs= -1)
    clf.fit(X_train_tfidf, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
       
    y_true, y_pred = y_test, clf.predict(X_test_tfidf)
    print("Accuracy of SVC on holdout data: ",accuracy_score(y_true,y_pred))
    print()
    
    print(classification_report(y_test, y_pred))
    print()

print('Running the model with best parameters')

scores = cross_val_score(clf.best_estimator_, X_train_tfidf, y_train, cv = 5)

print('Accuracy of model on cross validation dataset while training')
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))
y_pred = cross_val_predict(clf.best_estimator_, X_train_tfidf, y_train, cv = 5)
df = pd.DataFrame({'prediction':y_pred, 'obsevred': y_train})
confusion_matrix = ConfusionMatrix(df.obsevred, df.prediction)
#print("Confusion matrix:\n%s" % confusion_matrix)
#confusion_matrix.print_stats()
print('Confusion matrix showing classification on cross validation dataset while training')
confusion_matrix.plot(normalized=False, backend='seaborn', cmap='Blues',annot= True)
plt.show()

print('Best paraamters for the algorithm\n')
print (clf.best_estimator_)
best_svc = clf.best_estimator_
y_true, y_pred = y_test, clf.predict(X_test_tfidf)

print("Accuracy of SVC on holdout data: ",accuracy_score(y_true,y_pred))

df = pd.DataFrame({'prediction':y_pred, 'obsevred': y_test})
confusion_matrix = ConfusionMatrix(df.obsevred, df.prediction)
#print("Confusion matrix:\n%s" % confusion_matrix)
#confusion_matrix.print_stats()
confusion_matrix.plot(normalized=False, backend='seaborn', cmap='Blues',annot= True, fmt='.0f')
plt.savefig(data_path+'/Best_model.png', dpi = 600)
plt.show()

probs = pd.DataFrame(clf.predict_proba(X_test_tfidf), columns=clf.classes_)
arr = np.argsort(-probs.values, axis=1)
df1 = pd.DataFrame(probs.columns[arr], index=probs.index)
df1.columns = ['rank_'+ str(col)  for col in df1.columns]
final_svc = pd.concat([df.reset_index(0), probs], axis=1)
final_svc = pd.concat([df.reset_index(0), probs, df1], axis=1)
def correct_classification(c):
    if c.obsevred ==  c.prediction:
        return 1
    else:
        return 0
final_svc['correct_prediction'] = final_svc.apply(correct_classification, axis=1)

def matching_two_methods(c):
    if c.prediction ==  c['rank_0']:
        return 1
    else:
        return 0

final_svc['method_match'] = final_svc.apply(matching_two_methods, axis=1)


def matching_rank_2(c):
    if c.method_match ==  0:
        if c.obsevred ==  c['rank_1']:
            return 1
        else:
            return 0 
    else:
        return 0

final_svc['match_rank_2'] = final_svc.apply(matching_rank_2, axis=1)

def adjust_prediction(c):
    if c.method_match ==  0:
        return c['rank_1']
    else:
        return c['prediction']

final_svc['prediction_adjusted'] = final_svc.apply(adjust_prediction, axis=1)


final_svc.to_csv('C:/Users/Falco/Desktop/directory/WRMD_paper/outputs/final_figures/final_svc.csv')

print ('Adjusting the predictions\nby comparing class wise prediction probabilities and predicted classes\n')

print("Accuracy of SVC on holdout data: ",accuracy_score(final_svc['obsevred'],final_svc['prediction_adjusted']))
print()
df = final_svc[['obsevred','prediction_adjusted']]
confusion_matrix = ConfusionMatrix(df.obsevred, df.prediction_adjusted)
#print("Confusion matrix:\n%s" % confusion_matrix)
#confusion_matrix.print_stats()
confusion_matrix.plot(normalized=False, backend='seaborn', cmap='Blues',annot= True, fmt='.0f')
plt.savefig('C:/Users/Falco/Desktop/directory/WRMD_paper/outputs/final_figures/Final_model.png', dpi = 600)
cm = confusion_matrix.to_dataframe()
cm.to_csv('C:/Users/Falco/Desktop/directory/WRMD_paper/outputs/final_figures/confusion_matrix.csv')
plt.show()


In [ ]:
import seaborn as sns
fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize =(6,6))
cm.replace(0, np.nan, inplace= True)
sns.heatmap(cm, cmap='GnBu', mask=cm.isnull(),annot=True, fmt='.0f')
plt.show()

In [ ]:
import seaborn as sns
fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize =(6,6))
cm.replace(0, np.nan, inplace= True)
sns.heatmap(cm, cmap='GnBu', annot=True, fmt='.0f')
plt.show()

In [ ]:
confusion_matrix.to_dataframe().fillna(0)

In [ ]:
confusion_matrix.to_dataframe().fillna(0).sum(axis = 1)

In [ ]:
cm_perc = confusion_matrix.to_dataframe().fillna(0)
cm_perc = (cm_perc.fillna(0).T/cm_perc.fillna(0).sum(axis = 1)).T
cm_perc.replace(0, np.nan, inplace= True)
fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize =(8,8))
sns.heatmap(cm_perc, cmap='GnBu', annot=True, fmt='.2f', annot_kws={"size": 11})
plt.tight_layout()
plt.savefig('C:/Users/Falco/Desktop/directory/WRMD_paper/outputs/final_figures/Figure2_confusion_matrix.png', dpi = 600)
plt.savefig('C:/Users/Falco/Desktop/directory/WRMD_paper/outputs/final_figures/Figure2_confusion_matrix.svg', dpi = 600)

plt.show()

In [ ]:
cm_perc = confusion_matrix.to_dataframe().fillna(0)
cm_perc = (cm_perc.fillna(0).T/cm_perc.fillna(0).sum(axis = 1)).T
cm_perc.replace(0, np.nan, inplace= True)
fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize =(8*0.66,8*0.66))
sns.heatmap(cm_perc, cmap='GnBu', annot=True, fmt='.2f', annot_kws={"size": 11*0.66})
plt.tight_layout()
plt.savefig('C:/Users/Falco/Desktop/directory/WRMD_paper/outputs/final_figures/Figure2_confusion_matrix_v2.png', dpi = 600)
plt.savefig('C:/Users/Falco/Desktop/directory/WRMD_paper/outputs/final_figures/Figure2_confusion_matrix_v2.svg', dpi = 600)

plt.show()

In [ ]:
cm_perc.sum(axis=0)

In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
import seaborn as sns
lw = 2

In [ ]:
colors = ['#1b9e77', '#d95f02', '#7570b3']

In [ ]:
lw = 4
def plot_ROC(start, end , ax, fpr, tpr, roc_auc):
    colors = ['#1b9e77', '#d95f02', '#7570b3','#e7298a']
    for i, color, s in zip(range(start, end), colors, clf.classes_[start:end]):
        name = 'ROC ' + clf.classes_[i]+'(area = {1:0.2f})'
        ax.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='{0} (AUC = {1:0.2f})'
                 ''.format(s, roc_auc[i]))

    ax.plot([0, 1], [0, 1], 'k--', lw=lw)
    ax.set_xlim([-0.05, 1.0])
    ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc="lower right")

In [ ]:
n_classes = len(np.unique(y_train))
probs = clf.predict_proba(X_test_tfidf)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test, probs[:, i], pos_label = clf.classes_[i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
#fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), probs.ravel())
#roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
cmap = sns.color_palette("Set2", 12)

plt.rcParams['figure.figsize'] = (6,6)
# Plot all ROC curves
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, nrows=1, figsize =(18,6))
plot_ROC(start = 0, end = 4 , ax = ax1, fpr = fpr, tpr =tpr , roc_auc = roc_auc)
plot_ROC(start = 4, end = 8 , ax = ax2, fpr = fpr, tpr =tpr , roc_auc = roc_auc)
plot_ROC(start = 8, end = 12 , ax = ax3, fpr = fpr, tpr =tpr , roc_auc = roc_auc)
plt.savefig('C:\Users\Falco\Desktop\directory\WRMD_paper\outputs\ROC_curves.png', dpi = 600)
plt.show()

In [ ]:
clf

In [ ]:
def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)


def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df


def top_mean_feats(X_train_tfidf, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = X_train_tfidf[grp_ids].toarray()
    else:
        D = X_train_tfidf.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [ ]:
def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df['label'] = label
        dfs.append(feats_df)
    return dfs

In [ ]:
dataframes = top_feats_by_class(Xtr= X_test_tfidf, y= y_test, features=features)

In [ ]:
dataframes[0].head()

In [ ]:
clf.classes_

In [ ]:
for df in dataframes:
    df.set_index('feature').plot(kind = 'barh', title = df.label.unique()[0])

## Prediction on historical dataset

In [ ]:
%%time
predhist = clf.predict(X_hist_tfidf)
probs_hist = pd.DataFrame(clf.predict_proba(X_hist_tfidf), columns=clf.classes_, index= h_data.index)
arr = np.argsort(-probs_hist.values, axis=1)

In [ ]:
df2 = pd.DataFrame(probs_hist.columns[arr], index=h_data.index)
df2.columns = ['rank_'+ str(col)  for col in df2.columns]
df2.head()

In [ ]:
%% time
hist_final = h_data
hist_final['prediction'] =predhist 
hist_final = pd.concat([hist_final, probs_hist, df2], axis=1)
hist_final['method_match'] = hist_final.apply(matching_two_methods, axis=1)

def adjust_prediction(c):
    if c.method_match ==  0:
        return c['rank_0']
    else:
        return c['prediction']

final_svc['prediction_adjusted'] = final_svc.apply(adjust_prediction, axis=1)

hist_final['prediction_adjusted'] = hist_final.apply(adjust_prediction, axis=1)

In [ ]:
%% time
hist_final.to_csv(data_path+'/Predicted_final.csv')

## Final fit with all training data

In [ ]:
data.columns

In [ ]:
def split_into_tokens(message):
    message = unicode(message, 'utf8')  # convert bytes into proper unicode
    return TextBlob(message).words


def split_into_lemmas(message):
    message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]


def bag_of_words_complete_model(data_frame):
    print('count vectorize')
    count_vect = CountVectorizer(analyzer=split_into_lemmas,ngram_range= (1, 3), encoding='utf8',stop_words =None)
    X_counts = count_vect.fit_transform(data_frame['Condition'])
    
    print('tfid tranformer')
    tfidf_transformer = TfidfTransformer()
    X_tfidf = tfidf_transformer.fit_transform(X_counts)
    
    print('Training model on complete training dataset')
    print (X_tfidf.shape)
    
    return X_tfidf, data_frame['condition_predict'], count_vect, tfidf_transformer

best_svc = SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
                  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
                  max_iter=-1, probability=True, random_state=None, shrinking=True,
                  tol=0.001, verbose=False)

In [ ]:
def split_into_lemmas(message):
    message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

from sklearn.pipeline import Pipeline

count_vect = CountVectorizer(analyzer=split_into_lemmas,ngram_range= (1, 3), encoding='utf8',stop_words =None)
tfidf_transformer = TfidfTransformer()
text_clf = Pipeline([('vect', count_vect), ('tdif', tfidf_transformer), ('clf', best_svc)])


In [ ]:
%%time
text_clf.fit(X=data['Condition'], y=data['condition_predict'])

In [ ]:
_ = joblib.dump(text_clf, 'classification_pipeline.pkl')

In [ ]:
%%time
X_tfidf, Y, count_vectrotize_obj_complete , tfidf_transformer_object= bag_of_words_complete_model(data_frame= data)

In [ ]:
%%time
best_svc = SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
                  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
                  max_iter=-1, probability=True, random_state=None, shrinking=True,
                  tol=0.001, verbose=False)
best_svc.fit(X=X_tfidf, y=Y)

In [ ]:
best_svc

## Dumping the model into a .pkl file for later use


In [ ]:
from sklearn.externals import joblib
joblib.dump(best_svc, 'Syndrom_classification_model.pkl')

In [ ]:
joblib.dump(count_vectrotize_obj_complete, 'count_vectrotize_obj_complete.pkl')
joblib.dump(tfidf_transformer_object, 'tfidf_transformer_object.pkl')